<a href="https://colab.research.google.com/github/CHO-MINJUNG/BigContest2020/blob/master/%EB%B3%80%EC%88%98_%EC%83%9D%EC%84%B1_%EC%83%81%EB%8C%80_%EC%B5%9C%EA%B7%BC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

In [3]:
from google.colab import auth
auth.authenticate_user()

from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [4]:
teamhitter_vs = pd.read_csv('./gdrive/My Drive/빅콘테스트2020/1차_최종_데이터/1차_최종_상대_팀타자.csv', encoding='utf-8-sig')
teampitcher_vs = pd.read_csv('./gdrive/My Drive/빅콘테스트2020/1차_최종_데이터/1차_최종_상대_팀투수.csv', encoding='utf-8-sig')

In [13]:
teamhitter_vs = teamhitter_vs[teamhitter_vs['득점']<=15]
teamhitter_vs = teamhitter_vs.drop(labels = ['상대OBP',
       '상대SLG', '상대OPS', '상대GPA', '상대타율', '상대ISO', '상대BABIP', '상대HR%', '상대K%',
       '상대BB%', '상대BBK', '상대wOBA', '상대wRAA', '상대wRC', '상대RC', '상대RC27', '상대XR',
       '상대SPD', '상대SECA', '상대PSN'], axis=1)

In [14]:
teamhitter_vs.columns

Index(['게임키', '일자', '팀코드', '상대팀코드', '더블헤더코드', '초말', '타자', '타수', '타점', '득점',
       '안타', '2루타', '3루타', '홈런', '도루', '도루실패', '희타', '희비', '4구', '고4', '사구',
       '삼진', '병살타', '실책', '잔루', '득점권타율', '득점권타수', '득점권안타', '시즌', '단타'],
      dtype='object')

In [15]:
def runBaseP(row):
    previous = teamhitter_vs[(teamhitter_vs['시즌'] == row['시즌']) & (teamhitter_vs['팀코드'] == row['팀코드']) & (teamhitter_vs['상대팀코드'] == row['상대팀코드']) & (teamhitter_vs['일자'] < row['일자'])].tail(3)
    if len(previous) == 0:
        row['상대최근OBP'] = 0
        return row
    previous_sum = previous.sum()
    up = previous_sum['안타'] + previous_sum['4구']  + previous_sum['사구']
    down = previous_sum['사구'] + previous_sum['타수'] + previous_sum['4구'] + previous_sum['희비']
    if down == 0:
        row['상대최근OBP'] = 0
        return row
    row['상대최근OBP'] = up / down
    return row

def longHitP(row):
    previous = teamhitter_vs[(teamhitter_vs['시즌'] == row['시즌']) & (teamhitter_vs['팀코드'] == row['팀코드']) & (teamhitter_vs['상대팀코드'] == row['상대팀코드']) & (teamhitter_vs['일자'] < row['일자'])].tail(3)
    if len(previous) == 0:
        row['상대최근SLG'] = 0
        return row
    previous_sum = previous.sum()
    up = previous_sum['단타'] * 1 + previous_sum['2루타'] * 2 + previous_sum['3루타'] * 3 + previous_sum['홈런'] * 4
    down = previous_sum['타수']
    if down == 0:
        row['상대최근SLG'] = 0
        return row
    row['상대최근SLG'] = up / down

    return row
def hitP(row):
    previous = teamhitter_vs[(teamhitter_vs['시즌'] == row['시즌']) & (teamhitter_vs['팀코드'] == row['팀코드']) & (teamhitter_vs['상대팀코드'] == row['상대팀코드']) & (teamhitter_vs['일자'] < row['일자'])].tail(3)
    if len(previous) == 0:
        row['상대최근타율'] = 0
        return row
    previous_sum = previous.sum()
    up = previous_sum['안타']
    down = previous_sum['타수']
    if down == 0:
        row['상대최근타율'] = 0
        return row
    row['상대최근타율'] = up / down
    return row

def createBABIP(row):
  previous = teamhitter_vs[(teamhitter_vs['시즌'] == row['시즌']) & (teamhitter_vs['팀코드'] == row['팀코드']) & (teamhitter_vs['상대팀코드'] == row['상대팀코드']) & (teamhitter_vs['일자'] < row['일자'])].tail(3)
  if len(previous) == 0:
      row['상대최근BABIP'] = 0
      return row
  previous_sum = previous.sum()
  up = previous_sum['안타'] - previous_sum['홈런']
  down = previous_sum['타수'] - previous_sum['홈런'] - previous_sum['삼진'] + previous_sum['희비']
  if down == 0:
      row['상대최근BABIP'] = 0
      return row
  row['상대최근BABIP'] = up / down
  return row

def createBBK(row):
    previous = teamhitter_vs[(teamhitter_vs['시즌'] == row['시즌']) & (teamhitter_vs['팀코드'] == row['팀코드']) & (teamhitter_vs['상대팀코드'] == row['상대팀코드']) & (teamhitter_vs['일자'] < row['일자'])].tail(3)
    if len(previous) == 0:
        row['상대최근BB/K'] = 0
        return row
    previous_sum = previous.sum()
    up = previous_sum['4구']
    down = previous_sum['삼진']
    if down == 0:
        row['상대최근BB/K'] = 0
        return row
    row['상대최근BB/K'] = up / down
    return row

def createK_BB(row):
    previous = teamhitter_vs[(teamhitter_vs['시즌'] == row['시즌']) & (teamhitter_vs['팀코드'] == row['팀코드']) & (teamhitter_vs['상대팀코드'] == row['상대팀코드']) & (teamhitter_vs['일자'] < row['일자'])].tail(3)
    if len(previous) == 0:
        row['상대최근K%'] = 0
        row['상대최근BB%'] = 0
        return row
    previous_sum = previous.sum()
    up_k = previous_sum['삼진']
    up_BB = previous_sum['4구'] - previous_sum['고4']
    down = previous_sum['타자'] - previous_sum['고4']
    if down == 0:
        row['상대최근K%'] = 0
        row['상대최근BB%'] = 0
        return row
    row['상대최근K%'] = up_k / down
    row['상대최근BB%'] = up_BB / down
    return row

In [18]:
teamhitter_vs = teamhitter_vs.apply(runBaseP,axis=1)
teamhitter_vs = teamhitter_vs.apply(longHitP,axis=1)
teamhitter_vs['상대최근ops'] = teamhitter_vs['상대최근OBP'] + teamhitter_vs['상대최근SLG']
teamhitter_vs = teamhitter_vs.apply(hitP,axis=1)
teamhitter_vs['상대최근ISO'] = teamhitter_vs['상대최근SLG'] - teamhitter_vs['상대최근타율']
teamhitter_vs = teamhitter_vs.apply(createBABIP,axis=1)
teamhitter_vs = teamhitter_vs.apply(createBBK,axis=1)
teamhitter_vs = teamhitter_vs.apply(createK_BB,axis=1)

In [19]:
c_dic = {2020: [0.730, 0.759,	0.938, 1.284,	1.595,	1.959],
         2019: [0.721,	0.752,	0.945,	1.317,	1.653,	2.074],
         2018: [0.726,	0.753,	0.920,	1.242,	1.533,	1.857],
         2017: [0.728,	0.756,	0.927,	1.259,	1.558,	1.901],
         2016: [0.747,	0.774,	0.944,	1.273,	1.569,	1.898]}

c_dic2 = {2020: [0.345,	1.153, 4328, 56881],
         2019: [0.335,	1.242, 6548, 55612],
         2018: [0.351,	1.075, 7994, 56903],
         2017: [0.351,	1.107, 7682, 56881],
         2016: [0.362,	1.097, 8074, 57614]}

In [20]:
teamhitter_vs

,게임키,일자,팀코드,상대팀코드,더블헤더코드,초말,타자,타수,타점,득점,안타,2루타,3루타,홈런,도루,도루실패,희타,희비,4구,고4,사구,삼진,병살타,실책,잔루,득점권타율,득점권타수,득점권안타,시즌,단타,상대최근OBP,상대최근SLG,상대최근ops,상대최근타율,상대최근ISO,상대최근BABIP,상대최근BB/K,상대최근K%,상대최근BB%
0,20160401HHLG0,2016-04-01,LG,HH,0,B,47,42,4,5,9,2,0,1,2,1,1,0,4,0,0,11,0,0,8,0.333,12,4,2016,6,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,20160401HHLG0,2016-04-01,HH,LG,0,T,52,46,4,4,13,2,0,0,0,0,3,0,3,0,0,10,1,2,12,0.200,15,3,2016,11,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,20160401HTNC0,2016-04-01,NC,HT,0,B,36,30,5,5,9,2,0,2,0,0,1,0,5,0,0,9,1,1,7,0.143,7,1,2016,5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,20160401HTNC0,2016-04-01,HT,NC,0,T,38,34,3,4,8,3,0,1,0,0,0,0,3,0,1,10,1,0,7,0.100,10,1,2016,4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,20160401KTSK0,2016-04-01,SK,KT,0,B,36,36,4,4,10,5,0,1,0,0,0,0,0,0,0,7,1,1,5,0.375,8,3,2016,4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6395,20200719LTSS0,2020-07-19,LT,SS,0,T,31,29,2,2,6,1,0,1,0,0,1,0,1,0,0,7,3,0,2,0.000,2,0,2020,4,0.353448,0.423077,0.776525,0.288462,0.134615,0.314607,0.714286,0.119658,0.085470
6396,20200719OBHT0,2020-07-19,HT,OB,0,B,38,32,4,4,8,0,1,0,0,0,0,0,6,0,0,5,1,0,7,0.571,7,4,2020,7,0.370370,0.410526,0.780897,0.284211,0.126316,0.320988,0.923077,0.119266,0.100917
6397,20200719OBHT0,2020-07-19,OB,HT,0,T,41,37,8,8,11,3,0,1,2,1,0,0,2,1,2,4,0,0,6,0.364,11,4,2020,7,0.377193,0.336735,0.713928,0.295918,0.040816,0.329545,0.916667,0.103448,0.094828
6398,20200719WOSK0,2020-07-19,SK,WO,0,B,37,30,3,4,6,0,0,0,0,0,0,0,7,0,0,10,0,0,9,0.250,8,2,2020,6,0.297297,0.393939,0.691237,0.222222,0.171717,0.250000,0.416667,0.216216,0.090090


In [23]:
def createwOBA(row):
    previous = teamhitter_vs[(teamhitter_vs['시즌'] == row['시즌']) & (teamhitter_vs['팀코드'] == row['팀코드'])& (teamhitter_vs['상대팀코드'] == row['상대팀코드']) & (teamhitter_vs['일자'] < row['일자'])].tail(3)
    if len(previous) == 0:
        row['상대최근wOBA'] = 0
        return row
    previous_sum = previous.sum()
    ct = c_dic[row['시즌']]
    up = (ct[0] * (previous_sum['4구'] - previous_sum['고4']) + ct[1] * previous_sum['사구'] + ct[2] * previous_sum['단타'] + ct[3] * previous_sum['2루타'] + ct[4] * previous_sum['3루타'] + ct[5] * previous_sum['홈런'] )
    down = (previous_sum['타수'] + previous_sum['4구'] - previous_sum['고4'] + previous_sum['사구'] + previous_sum['희비'])
    if down == 0:
        row['상대최근wOBA'] = 0
        return row
    row['상대최근wOBA'] = up / down
    return row

def createwRAA(row):
    previous = teamhitter_vs[(teamhitter_vs['시즌'] == row['시즌']) & (teamhitter_vs['팀코드'] == row['팀코드'])& (teamhitter_vs['상대팀코드'] == row['상대팀코드']) & (teamhitter_vs['일자'] < row['일자'])].tail(3)
    if len(previous) == 0:
        row['상대최근wRAA'] = 0
        return row
    previous_sum = previous.sum()
    ct = c_dic2[row['시즌']]
    up = row['상대최근wOBA'] - ct[0]
    down = ct[1]
    row['상대최근wRAA'] = up / down * previous_sum['타자']
    return row

def createwRC(row):
    previous = teamhitter_vs[(teamhitter_vs['시즌'] == row['시즌']) & (teamhitter_vs['팀코드'] == row['팀코드'])& (teamhitter_vs['상대팀코드'] == row['상대팀코드']) & (teamhitter_vs['일자'] < row['일자'])].tail(3)
    if len(previous) == 0:
        row['상대최근wRC'] = 0
        return row
    previous_sum = previous.sum()
    ct = c_dic2[row['시즌']]
    up = ct[2]
    down = ct[3]
    row['상대최근wRC'] = (up / down) * previous_sum['타자'] + row['상대최근wRAA']
    return row

def createHR(row):
    previous = teamhitter_vs[(teamhitter_vs['시즌'] == row['시즌']) & (teamhitter_vs['팀코드'] == row['팀코드'])& (teamhitter_vs['상대팀코드'] == row['상대팀코드']) & (teamhitter_vs['일자'] < row['일자'])].tail(3)
    if len(previous) == 0:
        row['상대최근HR%'] = 0
        return row
    previous_sum = previous.sum()
    up_hr = previous_sum['홈런']
    down = previous_sum['타자'] - previous_sum['고4']
    if down == 0:
        row['상대최근HR%'] = 0
        return row
        
    row['상대최근HR%'] = up_hr / down
    return row

In [24]:
teamhitter_vs = teamhitter_vs.apply(createwOBA, axis=1)
teamhitter_vs = teamhitter_vs.apply(createwRAA, axis=1)
teamhitter_vs = teamhitter_vs.apply(createwRC, axis=1)
teamhitter_vs = teamhitter_vs.apply(createHR, axis=1)
teamhitter_vs['상대최근GPA'] = (1.8 * teamhitter_vs['상대최근OBP'] + teamhitter_vs['상대최근SLG']) / 4 

In [30]:
def createRC(row):
    previous = teamhitter_vs[(teamhitter_vs['시즌'] == row['시즌']) & (teamhitter_vs['팀코드'] == row['팀코드'])& (teamhitter_vs['상대팀코드'] == row['상대팀코드']) & (teamhitter_vs['일자'] < row['일자'])].tail(3)
    if len(previous) == 0:
        row['상대최근RC'] = 0
        return row
    previous_sum = previous.sum()
    upA = previous_sum['안타']+previous_sum['4구']-previous_sum['도루실패']+previous_sum['사구']-previous_sum['병살타']
    upB = (previous_sum['단타']*1.125)+(1.69*previous_sum['2루타'])+(3.02*previous_sum['3루타'])+(3.73*previous_sum['홈런'])+ 0.29*(previous_sum['4구']-previous_sum['고4']+previous_sum['사구'])
    down = previous_sum['타수']+previous_sum['4구']+previous_sum['사구']+previous_sum['희비']+previous_sum['희타']
    if down == 0:
        row['상대최근RC'] = 0
        return row
    row['상대최근RC'] = ((2.4*down+upA)*(3*down+upB)/(9*down)) - (0.9*down)
    
    return row

def createRC27(row):
    previous = teamhitter_vs[(teamhitter_vs['시즌'] == row['시즌']) & (teamhitter_vs['팀코드'] == row['팀코드'])& (teamhitter_vs['상대팀코드'] == row['상대팀코드']) & (teamhitter_vs['일자'] < row['일자'])].tail(3)   
    if len(previous) == 0:
        row['상대최근RC27'] = 0
        return row
    previous_sum = previous.sum()
    down = previous_sum['타수'] - previous_sum['안타']+previous_sum['병살타']+previous_sum['도루실패']+previous_sum['희타']+previous_sum['희비']
    if down == 0:
        row['상대최근RC27'] = 0
        return row
    row['상대최근RC27'] = (row['상대최근RC']/down)*27
    
    return row

In [31]:
teamhitter_vs = teamhitter_vs.apply(createRC, axis=1)
teamhitter_vs = teamhitter_vs.apply(createRC27, axis=1)

In [33]:
def XR(row):
    previous = teamhitter_vs[(teamhitter_vs['시즌'] == row['시즌']) & (teamhitter_vs['팀코드'] == row['팀코드'])& (teamhitter_vs['상대팀코드'] == row['상대팀코드']) & (teamhitter_vs['일자'] < row['일자'])].tail(3)
    if len(previous) == 0:
        row['상대최근XR'] = 0
        return row
    previous_sum = previous.sum()
    try:
        Y1 = 0.5*previous_sum['단타']+0.72*previous_sum['2루타']+1.04*previous_sum['3루타']+1.44*previous_sum['홈런']
        Y2 = (previous_sum['사구']+previous_sum['4구']-previous_sum['고4'])*0.34+0.25*previous_sum['고4']+0.18*previous_sum['도루']-0.32*previous_sum['도루실패']
        Y3 = -(previous_sum['타수']-previous_sum['안타']-previous_sum['삼진'])*0.09-0.098*previous_sum['삼진']-0.37*previous_sum['병살타']+0.37*previous_sum['희비']+0.04*previous_sum['희타']
    except ZeroDivisionError:
        row['상대최근XR'] = 0
        return row
    row['상대최근XR'] = Y1+Y2+Y3
    return row

def SPD(row):
    previous = teamhitter_vs[(teamhitter_vs['시즌'] == row['시즌']) & (teamhitter_vs['팀코드'] == row['팀코드'])& (teamhitter_vs['상대팀코드'] == row['상대팀코드']) & (teamhitter_vs['일자'] < row['일자'])].tail(3)
    if len(previous) == 0:
        row['상대최근SPD'] = 0
        return row
    previous_sum = previous.sum()
    try:
        f1 = ((previous_sum['도루']+3)/(previous_sum['도루']+previous_sum['도루실패']+7)-0.4) * 20
        f2 = ((previous_sum['도루']+previous_sum['도루실패'])/(previous_sum['단타']+previous_sum['4구']+previous_sum['사구']))**0.5/0.07
        f3 = previous_sum['3루타']/(previous_sum['타수']-previous_sum['홈런']-previous_sum['삼진'])/ 0.02*10
        f4 = ((previous_sum['득점']-previous_sum['홈런'])/(previous_sum['안타']+previous_sum['4구']-previous_sum['홈런']-previous_sum['사구'])-0.1)/0.04
        f5 = (0.055-previous_sum['병살타']/(previous_sum['타수']-previous_sum['홈런']-previous_sum['삼진']))/0.005
    except ZeroDivisionError:
        row['상대최근SPD'] = 0
        return row
    row['상대최근SPD'] = (f1+f2+f3+f4+f5) / 5
    return row


In [35]:
teamhitter_vs = teamhitter_vs.apply(XR, axis=1)
teamhitter_vs = teamhitter_vs.apply(SPD, axis=1)

In [40]:
def SecA(row):
    previous = teamhitter_vs[(teamhitter_vs['시즌'] == row['시즌']) & (teamhitter_vs['팀코드'] == row['팀코드'])& (teamhitter_vs['상대팀코드'] == row['상대팀코드']) & (teamhitter_vs['일자'] < row['일자'])].tail(3)
    if len(previous) == 0:
        row['상대최근SecA'] = 0
        return row
    previous_sum = previous.sum()
    up = previous_sum['4구']+(previous_sum['2루타']+previous_sum['3루타']*2+previous_sum['홈런']*3)+previous_sum['도루']-previous_sum['도루실패']
    down = previous_sum['타수']
    if down == 0:
        row['상대최근SecA'] = 0
        return row
    row['상대최근SecA'] = up/down
    return 
    
def psn(row):
    previous = teamhitter_vs[(teamhitter_vs['시즌'] == row['시즌']) & (teamhitter_vs['팀코드'] == row['팀코드'])& (teamhitter_vs['상대팀코드'] == row['상대팀코드']) & (teamhitter_vs['일자'] < row['일자'])].tail(3)
    if len(previous) == 0:
        row['상대최근PSN'] = 0
        return row
    previous_sum = previous.sum()
    try:
        psn = 2*(previous_sum["홈런"]*previous_sum["도루"])/(previous_sum["홈런"]+previous_sum["도루"])
    except ZeroDivisionError:
        psn = 0
    row['상대최근PSN']= psn
    return row

In [37]:
teamhitter_vs = teamhitter_vs.apply(SecA, axis=1)

In [41]:
teamhitter_vs = teamhitter_vs.apply(psn, axis=1)

In [42]:
teamhitter_vs.to_csv('./gdrive/My Drive/빅콘테스트2020/2차_최종_최근_상대_팀타자.csv', encoding='utf-8-sig', index=False)